# **Análisis de Churn de Clientes**

Este proyecto aborda el desafío de identificar los factores que influyen en la cancelación de servicios por parte de los clientes. Utiliza datos reales extraídos desde una API en formato JSON, aplicando técnicas de ETL para su transformación y análisis. A través de visualizaciones estratégicas y un análisis exploratorio profundo, se generan insights clave para mejorar la retención de clientes.

## 1. Configuración del Entorno
Primero, importamos las librerías necesarias para el análisis y la visualización de datos.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json

## 2. Carga y Transformación Inicial de Datos
En este paso, cargamos los datos desde el archivo JSON y los transformamos en un DataFrame de Pandas. La estructura del JSON es anidada, por lo que utilizaremos pd.json_normalize para aplanarla y facilitar su manipulación.

In [5]:
# Cargar el archivo JSON
file_path = 'data/TelecomX_Data.json'
with open(file_path, 'r') as f:
    data = json.load(f)

# Verificar la estructura del JSON para asegurar el aplanamiento correcto
if isinstance(data, dict) and 'customers' in data:
    customers = data['customers']
else:
    customers = data

# Aplanar todos los campos anidados en un solo DataFrame
df = pd.json_normalize(customers, sep='.')

# Renombrar columnas para mayor claridad
df = df.rename(columns={
    'customerID': 'customerID',
    'Churn': 'Churn',
    'customer.gender': 'Gender',
    'customer.SeniorCitizen': 'SeniorCitizen',
    'customer.Partner': 'Partner',
    'customer.Dependents': 'Dependents',
    'customer.tenure': 'Tenure',
    'phone.PhoneService': 'PhoneService',
    'phone.MultipleLines': 'MultipleLines',
    'internet.InternetService': 'InternetService',
    'internet.OnlineSecurity': 'OnlineSecurity',
    'internet.OnlineBackup': 'OnlineBackup',
    'internet.DeviceProtection': 'DeviceProtection',
    'internet.TechSupport': 'TechSupport',
    'internet.StreamingTV': 'StreamingTV',
    'internet.StreamingMovies': 'StreamingMovies',
    'account.Contract': 'Contract',
    'account.PaperlessBilling': 'PaperlessBilling',
    'account.PaymentMethod': 'PaymentMethod',
    'account.Charges.Monthly': 'MonthlyCharges',
    'account.Charges.Total': 'TotalCharges'
})

print("Datos cargados y transformados con éxito.")

Datos cargados y transformados con éxito.


## 3. Exploración Inicial y Verificación de Datos
Ahora, exploraremos el DataFrame para entender su estructura, los tipos de datos y si existen problemas evidentes como valores nulos o formatos incorrectos.

In [6]:
# Mostrar las primeras 5 filas para tener una vista previa
print("Vista previa de los datos:")
print(df.head())

# Obtener información general del DataFrame, incluyendo tipos de datos y valores no nulos
print("\nInformación general del DataFrame:")
df.info()

# Revisar si hay valores duplicados
print(f"\nNúmero de filas duplicadas: {df.duplicated().sum()}")

# La columna 'TotalCharges' se cargó como 'object'. Es un problema común que debemos corregir.
# Vamos a investigar por qué.
# Intentaremos convertirla a numérico, y los valores que fallen se marcarán como NaT (Not a Time) o NaN (Not a Number).
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Ahora, contemos cuántos valores nulos hay en 'TotalCharges' después de la conversión.
print(f"\nValores nulos en 'TotalCharges' después de la conversión: {df['TotalCharges'].isnull().sum()}")

# Veamos las filas donde 'TotalCharges' es nulo para entender la causa.
print("\nFilas con 'TotalCharges' nulos:")
print(df[df['TotalCharges'].isnull()][['Tenure', 'MonthlyCharges', 'TotalCharges']])

Vista previa de los datos:
   customerID Churn  Gender  SeniorCitizen Partner Dependents  Tenure  \
0  0002-ORFBO    No  Female              0     Yes        Yes       9   
1  0003-MKNFE    No    Male              0      No         No       9   
2  0004-TLHLJ   Yes    Male              0      No         No       4   
3  0011-IGKFF   Yes    Male              1     Yes         No      13   
4  0013-EXCHZ   Yes  Female              1     Yes         No       3   

  PhoneService MultipleLines InternetService  ... OnlineBackup  \
0          Yes            No             DSL  ...          Yes   
1          Yes           Yes             DSL  ...           No   
2          Yes            No     Fiber optic  ...           No   
3          Yes            No     Fiber optic  ...          Yes   
4          Yes            No     Fiber optic  ...           No   

  DeviceProtection TechSupport StreamingTV StreamingMovies        Contract  \
0               No         Yes         Yes              No 

**Observación Clave**: El análisis revela que los 11 valores nulos en TotalCharges corresponden a clientes con una antigüedad (Tenure) de 0. Esto es lógico, ya que son clientes nuevos que aún no han acumulado cargos totales.

## 4. Limpieza y Corrección de Datos
Con los problemas identificados, procedemos a limpiar el DataFrame. Corregiremos los valores nulos en TotalCharges y nos aseguraremos de que todos los tipos de datos sean correctos.

In [7]:
# Rellenamos los valores nulos en 'TotalCharges' con 0.
# Esto es apropiado porque los clientes con 0 meses de antigüedad no han generado cargos totales.
df['TotalCharges'].fillna(0, inplace=True)

# Verificamos que ya no hay valores nulos en la columna
print(f"Valores nulos restantes en 'TotalCharges': {df['TotalCharges'].isnull().sum()}")

# Ahora la columna 'SeniorCitizen' es numérica (0 o 1) pero representa una categoría.
# La convertiremos a un formato más claro ('No'/'Sí') para mejorar la legibilidad en las visualizaciones.
df['SeniorCitizen'] = df['SeniorCitizen'].map({0: 'No', 1: 'Yes'})

print("\nLimpieza de datos completada.")

Valores nulos restantes en 'TotalCharges': 0

Limpieza de datos completada.


C:\Users\apicn\AppData\Local\Temp\ipykernel_39452\3163623213.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalCharges'].fillna(0, inplace=True)
